# SarXarray producing STM input data

In [1]:
import numpy as np
from pathlib import Path
import sarxarray

### Load a SLC stack

In [2]:
# You many need to change this cell to your local data directory
path = Path('../../data/nl_veenweiden_s1_asc_t088/')

In [3]:
# Make a list of SLCs to read
f_slc = 'slc_srd_nl_veenweiden.raw' 
list_slcs = [p/f_slc for p in path.rglob("????????")]

# Metadata of the stack
shape=(9914, 41174)
dtype = np.dtype([('re', np.float32), ('im', np.float32)])

stack = sarxarray.from_binary(list_slcs, shape, dtype=dtype)
stack

<xarray.Dataset>
Dimensions:    (azimuth: 9914, range: 41174, time: 3)
Coordinates:
  * azimuth    (azimuth) int64 0 1 2 3 4 5 6 ... 9908 9909 9910 9911 9912 9913
  * range      (range) int64 0 1 2 3 4 5 ... 41168 41169 41170 41171 41172 41173
  * time       (time) int64 0 1 2
Data variables:
    complex    (azimuth, range, time) complex64 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>
    amplitude  (azimuth, range, time) float32 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>
    phase      (azimuth, range, time) float32 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>

### Append other fields

In [4]:
# Append h2ph 
f_h2ph = 'h2ph_nl_veenweiden.raw' 
list_h2phs = [p/f_h2ph for p in path.rglob("????????")]
h2phs = sarxarray.from_binary(list_h2phs, shape, vlabel="h2ph", dtype=np.float32)
stack = stack.merge(h2phs)
stack

<xarray.Dataset>
Dimensions:    (azimuth: 9914, range: 41174, time: 3)
Coordinates:
  * azimuth    (azimuth) int64 0 1 2 3 4 5 6 ... 9908 9909 9910 9911 9912 9913
  * range      (range) int64 0 1 2 3 4 5 ... 41168 41169 41170 41171 41172 41173
  * time       (time) int64 0 1 2
Data variables:
    complex    (azimuth, range, time) complex64 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>
    amplitude  (azimuth, range, time) float32 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>
    phase      (azimuth, range, time) float32 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>
    h2ph       (azimuth, range, time) float32 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>

In [6]:
# Append lat and lon
f_lat = [path/'phi_nl_veenweiden.raw']
f_lon = [path/'lam_nl_veenweiden.raw']
lat = sarxarray.from_binary(f_lat, shape, vlabel="lat", dtype=np.float32)
lon = sarxarray.from_binary(f_lon, shape, vlabel="lon", dtype=np.float32)
stack = stack.merge(lat)
stack = stack.merge(lon)
stack

<xarray.Dataset>
Dimensions:    (azimuth: 9914, range: 41174, time: 3)
Coordinates:
  * azimuth    (azimuth) int64 0 1 2 3 4 5 6 ... 9908 9909 9910 9911 9912 9913
  * range      (range) int64 0 1 2 3 4 5 ... 41168 41169 41170 41171 41172 41173
  * time       (time) int64 0 1 2
Data variables:
    complex    (azimuth, range, time) complex64 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>
    amplitude  (azimuth, range, time) float32 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>
    phase      (azimuth, range, time) float32 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>
    h2ph       (azimuth, range, time) float32 dask.array<chunksize=(5, 41174, 1), meta=np.ndarray>
    lat        (azimuth, range, time) float32 dask.array<chunksize=(5, 41174, 3), meta=np.ndarray>
    lon        (azimuth, range, time) float32 dask.array<chunksize=(5, 41174, 3), meta=np.ndarray>

### Export to a Space Time Matrix

In [7]:
stack_subset = stack.sel(azimuth=range(6000,6500), range=range(11000,11500))

In [8]:
stm = stack_subset.slcstack.point_selection(threshold=2, method="amplitude_dispersion")
stm

<xarray.Dataset>
Dimensions:    (points: 141269, time: 3)
Coordinates:
  * time       (time) int64 0 1 2
    azimuth    (points) int64 6000 6000 6000 6000 6000 ... 6499 6499 6499 6499
    range      (points) int64 11001 11003 11007 11009 ... 11493 11496 11497
  * points     (points) int64 0 1 2 3 4 5 ... 141264 141265 141266 141267 141268
Data variables:
    complex    (points, time) complex128 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    amplitude  (points, time) float32 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    phase      (points, time) float32 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    h2ph       (points, time) float32 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    lat        (points, time) float32 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    lon        (points, time) float32 dask.array<chunksize=(1000, 3), meta=np.ndarray>

In [9]:
# Export to Zarr
stm.to_zarr("stm.zarr")

In [10]:
# Load exported STM in Zarr
import xarray as xr
stm_read = xr.open_zarr("stm.zarr")
stm_read

<xarray.Dataset>
Dimensions:    (points: 141269, time: 3)
Coordinates:
    azimuth    (points) int64 dask.array<chunksize=(35318,), meta=np.ndarray>
  * points     (points) int64 0 1 2 3 4 5 ... 141264 141265 141266 141267 141268
    range      (points) int64 dask.array<chunksize=(35318,), meta=np.ndarray>
  * time       (time) int64 0 1 2
Data variables:
    amplitude  (points, time) float32 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    complex    (points, time) complex128 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    h2ph       (points, time) float32 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    lat        (points, time) float32 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    lon        (points, time) float32 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    phase      (points, time) float32 dask.array<chunksize=(1000, 3), meta=np.ndarray>